# 任务四

In [ ]:
# 导入数据
import pandas as pd
df1 = pd.read_excel(r'D:\研究生资料\玛氏数据\5-22\chengdu_521.xlsx')
df_POI = pd.read_excel(r'D:\研究生资料\玛氏数据\chengdu_poi_category.xlsx')

## 一、	巧克力/口香糖/糖果平均下单时间大于6个月门店对应的POI销量

1)	求巧克力平均下单时间大于6个月的门店；


In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

# 把Chocolate的那部分分成下单周期小于6个月和下达周期大于6个月两类
Cstore_id_da6 = [] # 用来储存买Chocolate的那部分下单周期大于6个月的'store_code'
Cstore_id_xiao6 = [] # 用来储存买Chocolate的那部分下单周期小于等于6个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da6.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            Cstore_id_da6.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao6.append(Chocolate_storeid[i]) 

2)	求出（1）中每个门店对应的POI，去重、去充电宝



In [ ]:
# Cstore_id_da6对应的poi
Cstore_id_da6_POI=df_POI[df_POI['storecode'].isin(Cstore_id_da6)]
# 去充电宝
Cstore_id_da6_POI=Cstore_id_da6_POI.drop(Cstore_id_da6_POI[Cstore_id_da6_POI['s_cat_value']=='充电宝'].index)
# 对每个门店的poi去重（相同POI留下距离近的那个POI）
Cstore_id_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k1_1 = Cstore_id_da6_POI.groupby(['storecode','s_cat_id']).head(1)

3)	求出（2）中每个门店巧克力的销量（case-ordered）


In [ ]:
# 在k1_1里面筛选出门店信息；
storeid=set(k1_1['storecode'])
storeid=list(storeid)
# 在表df1中筛选出storeid卖巧克力的门店
demo1=df1.loc[df1['store_code'].isin(storeid)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
storeid2=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(storeid2)):
    demo3 = df1.loc[df1['store_code']==storeid2[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

4)	将（3）中的每个门店销量赋值到3中每个门店对应的POI上


In [ ]:
# k1_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'storeid2':storeid2,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='storeid2',inplace=True)

k1_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['storeid2'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k1_1.loc[k1_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])



k1_1['总销量']=sum_3

fff=k1_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

5)	求每个POI的销量累计值，按照销量排序；

In [ ]:
k5_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k5_4 = k5_4.reset_index()
k5_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k5_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k5_4['b_cat_id']=r1
k5_4['b_cat_value']=r2
k5_4['m_cat_id']=r3
k5_4['m_cat_value']=r4

k5_4=k5_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

k5_4.to_excel(r'C:\Users\Administrator\Desktop\任务四（一）.xlsx')